#### Étape 1 : Traduction
Happy Transformer : https://happytransformer.com/

In [2]:
from happytransformer import HappyTextToText, TTSettings
from collections import defaultdict
from curl_cffi import requests
from bs4 import BeautifulSoup as soup
import json

def get_lang_tags(url):

    responce = requests.get(url, impersonate="firefox133")
    page = soup(responce.text, 'html.parser')
    table = page.find_all("td")

    dict_lang = defaultdict(str)
    for r in range(0, len(table)-1, 2):
        dict_lang[table[r].text] = table[r+1].text

    return dict_lang

url_lang = "https://developers.google.com/admin-sdk/directory/v1/languages"
lang_tags = get_lang_tags(url_lang)

# Sauvegarder le dictionnaire des langues en fichier json 
with open("lang_tags.json", "w") as file:
    json.dump(lang_tags, file, indent=4)

source = "en"
target = "fr"

model_LANG = f"Helsinki-NLP/opus-mt-{source}-{target}"

inputs = "What's the weather like today and tomorrow?"
settings = TTSettings(do_sample=True, top_k=50, temperature=0.7)
happy_tt = HappyTextToText("MARIAN", model_LANG)

outputs = happy_tt.generate_text(inputs, args=settings).text

print(outputs)

03/24/2025 16:33:05 - INFO - happytransformer.happy_transformer -   Using device: cpu
03/24/2025 16:33:05 - INFO - happytransformer.happy_transformer -   Moving model to cpu
03/24/2025 16:33:05 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


Quel temps font-ils aujourd'hui et demain ?


#### Étape 2 : Quantity Estimation
Pour une évaluation au niveau de la phrase, il existe deux architectures différentes : Mono et Siamese.

TransQuest : https://github.com/TharinduDR/TransQuest

In [10]:
import torch # Importer PyTorch
from transquest.algo.sentence_level.siamesetransquest.run_model import SiameseTransQuestModel # Importer modèle Siamese
from transquest.algo.sentence_level.monotransquest.run_model import MonoTransQuestModel # Importer modèle Mono

##### Mono

In [11]:
# Chargement du modèle pré-entraîné "monotransquest-da-multilingual".
model = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-multilingual", num_labels=1, use_cuda=torch.cuda.is_available())

# Prédiction de la qualité de traduction pour une paire de phrases.
pred, raw_output = model.predict([[inputs, outputs]])
print(pred, raw_output)

/home/guilhem/Documents/Master_TAL/Technique_Web/Venv_TransQuest/lib/python3.10/site-packages/transquest/algo/sentence_level/monotransquest/run_model.py:251: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  3.21it/s]

0.668252170085907 0.668252170085907


##### Siamese

In [ ]:
# Chargement du modèle pré-entraîné "siamesetransquest-da-multilingual".
model = SiameseTransQuestModel("TransQuest/siamesetransquest-da-multilingual")

# Prédiction de la qualité de traduction pour une paire de phrases.
pred = model.predict([[inputs, outputs]])
print(pred)